In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_sample_submission.csv
/kaggle/input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train.csv


KeyboardInterrupt: 

## Importing necessary libraries

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.segmentation import slic
from skimage import io
import cv2
from skimage.color import rgb2gray
from skimage.graph import rag_mean_color
from torch_geometric.data import Data
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_max_pool
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

## Loading data into dataframes

In [2]:
df = pd.read_csv('/kaggle/input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train.csv')

In [3]:
df.head()

,ID,Label
0,ID_12cadc6af_epidural,0
1,ID_12cadc6af_intraparenchymal,0
2,ID_12cadc6af_intraventricular,0
3,ID_12cadc6af_subarachnoid,0
4,ID_12cadc6af_subdural,0


In [4]:
df.sample(5)

,ID,Label
3397572,ID_1096b3ac9_epidural,0
3044744,ID_7d2a8f1ff_intraventricular,1
3993649,ID_bed47c2f5_intraparenchymal,0
385411,ID_08562832d_intraparenchymal,0
4255988,ID_982db8ca7_intraventricular,0


## Installing necessary libraries

In [5]:
!pip install torch torchvision torch-geometric scikit-image opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.1 MB/s eta 0:00:0000:010:01


## Image segmentation

In [8]:
# create image segments
def segment_image(image_path):
    # Load image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply SLIC segmentation
    segments = slic(image, n_segments=100, compactness=10)
    
    return segments

## Convert segments to graphs

In [23]:
def image_to_graph(image_path):
    image = io.imread(image_path)
    segments = segment_image(image_path)
    
    # Create RAG (Region Adjacency Graph)
    rag = rag_mean_color(image, segments)
    
    # Create node features (mean intensity in this case)
    node_features = []
    for region in rag.nodes:
        mask = segments == region
        mean_intensity = image[mask].mean()
        node_features.append([mean_intensity])
    
    node_features = torch.tensor(node_features, dtype=torch.float)
    
    edges = []
    for (node1, node2) in rag.edges:
        edges.append([node1, node2])
    
    edge_index = torch.tensor(np.array(edges).T, dtype=torch.long)
    
    # Create graph data object
    data = Data(x=node_features, edge_index=edge_index)
    
    return data

## Process all images

In [ ]:
def process_dataset(image_dir):
    graphs = []
    labels = []
    
    # List of classes
    classes = ['subdural', 'epidural', 'subarachnoid']
    
    for label, class_name in enumerate(classes):
        class_path = f"{image_dir}/{class_name}/"
        image_names = os.listdir(class_path)
        
        for image_name in image_names:
            image_path = f"{class_path}{image_name}"
            graph = image_to_graph(image_path)
            graphs.append(graph)
            labels.append(label)
    
    return graphs, labels

## Define the GCNN model

In [15]:
class GCNNModel(nn.Module):
    def __init__(self, input_features, hidden_channels):
        super(GCNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv6 = nn.Conv2d(512, 1024, 3, padding=1)

        self.conv7 = nn.Conv2d(1024, 2048, 3, padding=1)
        self.conv8 = nn.Conv2d(2048, 4096, 3, padding=1)
        self.conv9 = nn.Conv2d(4096, 8192, 3, padding=1)

        self.fc1 = nn.Linear(8192 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 3)


    def forward(self, data):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = self.pool(F.relu(self.conv6(x)))

        x = self.pool(F.relu(self.conv7(x)))
        x = self.pool(F.relu(self.conv8(x)))
        x = self.pool(F.relu(self.conv9(x)))

        x = x.view(-1, 8192 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


## Training the GCNN model

In [16]:
def train_gcnn(model, train_loader, optimizer, criterion, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}')     

## Evaluate the model

In [18]:
def evaluate_gcnn(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)
            all_labels.extend(data.y.cpu().numpy())
            all_preds.extend(pred.cpu().numpy())
    
            # Convert lists to numpy arrays
            all_labels = np.array(all_labels)
            all_preds = np.array(all_preds)
    
            # Calculate metrics
            accuracy = accuracy_score(all_labels, all_preds)
            f1 = f1_score(all_labels, all_preds, average='weighted')
            precision = precision_score(all_labels, all_preds, average='weighted')
            recall = recall_score(all_labels, all_preds, average='weighted')
            cm = confusion_matrix(all_labels, all_preds)
            print(f'Accuracy: {accuracy * 100:.2f}%')
            print(f'F1 Score: {f1:.2f}')
            print(f'Precision: {precision:.2f}')
            print(f'Recall: {recall:.2f}')
    
            # Print confusion matrix
            print('Confusion Matrix:')
            print(cm)

## Implementing the model and data

In [ ]:
# processing all images
graphs,labels = process_dataset('/kaggle/input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train')

In [ ]:
# creating a model instance
model = GCNNModel()

In [ ]:
# training the model
train_gcnn(model,graphs,labels)

In [ ]:
#evaluating the model
evaluate_gcnn(model,labels)